# Programação Matemática (GCC118)
## Universidade Federal de Lavras (UFLA)
### Instituto de Ciências Exatas e Tecnológicas

#### Grupo
- Ranulfo Mascari Neto
- Heitor Rodrigues Sabino

## Problema 3

A cidade de União Paulista enfrenta uma séria carência orçamentária. Em busca de uma solução de longo prazo, a câmara de vereadores da cidade aprova uma melhoria da base da cobrança de impostos que prevê a condenação de uma área habitacional do centro da cidade e sua substituição por um conjunto habitacional moderno.

O projeto envolve duas fases:

1. Demolição das casas que estão aquém do padrão para liberar terreno para o novo projeto;
2. Construção do novo conjunto urbano. A seguir, daremos um resumo da situação.

Um total de 300 casas aquém do padrão podem ser demolidas. Cada casa ocupa um lote de 0,25 acres. O custo de demolição de uma casa condenada é de $2.000. Os tamanhos de lotes para domicílios (unidades) simples, duplos, triplos e quádruplos são de 0,18; 0,28; 0,4 e 0,5 acres, respectivamente. Ruas, espaços abertos e instalações públicas ocupam 15% da área disponível.

No novo conjunto habitacional, as unidades triplas e quádruplas representam no mínimo 25% do total. Unidades simples devem representar no mínimo 20% de todas as unidades, e unidades duplas, no mínimo 10%. O imposto cobrado por unidade para unidades simples, duplas, triplas e quádruplas é de \$1.000, \$1.900, \$2.700 e \$3.400, respectivamente.

O custo de construção por unidade domiciliar simples, dupla, tripla e quádrupla é de \$50.000, \$70.000, \$130.000 e \$160.000, respectivamente. O financiamento acordado com um banco local será de no máximo \$15 milhões.

### Objetivo
Quantas unidades de cada tipo devem ser construídas para maximizar a arrecadação de impostos?

### Instalação da biblioteca PuLP

Para mais informações, acesse: [PuLP 2.9.0](https://pypi.org/project/PuLP/).

In [1]:
!pip install pulp
import pulp

## Modelagem Matemática

A seguir, apresentaremos a modelagem matemática deste problema, especificando os principais elementos da modelagem de um problema de programação matemática: $(i)$ parâmetros (dados); $(ii)$ variáveis de decisão; $(iii)$ modelagem, composta por uma função objetivo, restrições do problema e restrições de domínio das variáveis de decisão.

### Parâmetros

- **Tamanho do Lote**:
  - $ l_i \in \mathbb{R}_+ $: tamanho do lote do modelo de domicílio $ i $, onde $ i \in D $.
  - $ l_s \in \mathbb{R}_+ $: tamanho do lote de casas aquém do padrão.

- **Ocupação Mínima**:
  - $ o_i \in \mathbb{R}_+ $: porcentagem mínima de ocupação exigida para o modelo de domicílio $ i $, onde $ i \in D $.

- **Imposto**:
  - $ t_i \in \mathbb{R}_+ $: imposto gerado pelo modelo de domicílio $ i $, onde $ i \in D $.

- **Custo de Construção**:
  - $ c_i \in \mathbb{R}_+ $: custo de construção do modelo de domicílio $ i $, onde $ i \in D $.

- **Domicílios**:
  - $ D $: conjunto dos modelos de domicílios, onde $ i \in D $ representa:
    1. **Simples**:
      - Tamanho do Lote: 0,18 acres;
      - Ocupação Mínima: 20%;
      - Imposto: \$1.000;
      - Custo de Construção: \$50.000;
      
    2. **Duplos**:
      - Tamanho do Lote: 0,28 acres;
      - Ocupação Mínima: 10%;
      - Imposto: \$1.900;
      - Custo de Construção: \$70.000;

    3. **Triplos**:
      - Tamanho do Lote: 0,40 acres;
      - Ocupação Mínima: 25%;
      - Imposto: \$2.700;
      - Custo de Construção: \$130.000;

    4. **Quádruplos**:
      - Tamanho do Lote: 0,50 acres;
      - Ocupação Mínima: 25%;
      - Imposto: \$3.400;
      - Custo de Construção: \$160.000;

- **Total de Casas Demolidas**:
  - $ T \in \mathbb{N} $: número total de casas aquém do padrão que podem ser demolidas, definido como 300 casas.

- **Custo de Demolição**:
  - $ d_s \in \mathbb{R}_+ $: custo de demolição de uma casa condenada, definido como $2.000 por casa.

- **Financiamento**:
  - $ F \in \mathbb{R}_+ $: valor máximo do financiamento acordado com o banco local, definido como 15 milhões de dólares.

In [2]:
D = ["Simples", "Duplo", "Triplo", "Quadruplo"]

l = {
    "Simples": 0.18,
    "Duplo": 0.28,
    "Triplo": 0.4,
    "Quadruplo": 0.5
}

l_s = 0.25

o = {
    "Simples": 0.2,
    "Duplo": 0.1,
    "Triplo": 0.25,
    "Quadruplo": 0.25
}

t = {
    "Simples": 1000,
    "Duplo": 1900,
    "Triplo": 2700,
    "Quadruplo": 3400,
}

c = {
    "Simples": 50000,
    "Duplo": 70000,
    "Triplo": 130000,
    "Quadruplo": 160000,
}

T = 300

d_s = 2000

F = 15000000

### Declaração do objeto que representa o modelo matemático

In [3]:
modelo = pulp.LpProblem('arrecadacao_de_impostos', pulp.LpMaximize)

### Variáveis de Decisão

- $x_i \ge 0$: representa o número de unidades de domicílios a serem construídas do modelo $i$, onde $i \in D$.


In [4]:
x = pulp.LpVariable.dicts('x', D, lowBound=0, cat='Integer')

- $0 \le n \le T$: representa o número de casas aquém do padrão que serão demolidas.

In [5]:
n = pulp.LpVariable('n', cat='Integer', lowBound=0, upBound=T)

### Função Objetivo

- Maximização da Arrecadação de Impostos: $\max \sum_{i \in D} t_ix_i$

  onde:
  - $ x_i $ é o número de unidades do modelo de domicílio $ i $ construídas, onde $ i \in D $.
  - $ t_i $ é o imposto gerado por cada unidade do modelo de domicílio $ i $, onde $ i \in D $.


In [6]:
modelo += pulp.lpSum([t[i] * x[i] for i in D])

### Restrições

As restrições do problema são as seguintes:

- **Restrição de Lote**: Os tamanhos de lotes para domicílios (unidades) não podem ultrapassar a área disponível:

  $$\sum_{i \in D} l_i x_i \le (100\% - 15\%)l_s n$$

  considerando que 15% da área disponível é ocupado por ruas, espaços abertos e instalações públicas.

In [7]:
modelo += pulp.lpSum([l[i] * x[i] for i in D]) <= (1 - 0.15) * l_s * n

- **Restrição de Ocupação**: As respectivas unidades devem respeitar o limite mínimo do total de unidades:

  $$\frac{x_i}{ \sum_{j \in D} x_j} \ge o_i, \quad \forall i \in D$$



In [8]:
for i in D:
  modelo += x[i] >= o[i] * pulp.lpSum([x[j] for j in D])

- **Restrição de Financiamento**: As construções e demolições devem respeitar o limite máximo do financiamento acordado com o banco local:
  $$\sum_{i \in D} c_i x_i + d_s n \le F$$

In [9]:
modelo += pulp.lpSum([c[i] * x[i] for i in D]) + d_s * n <= F

In [10]:
modelo

arrecadacao_de_impostos:
MAXIMIZE
1900*x_Duplo + 3400*x_Quadruplo + 1000*x_Simples + 2700*x_Triplo + 0
SUBJECT TO
_C1: - 0.2125 n + 0.28 x_Duplo + 0.5 x_Quadruplo + 0.18 x_Simples
 + 0.4 x_Triplo <= 0

_C2: - 0.2 x_Duplo - 0.2 x_Quadruplo + 0.8 x_Simples - 0.2 x_Triplo >= 0

_C3: 0.9 x_Duplo - 0.1 x_Quadruplo - 0.1 x_Simples - 0.1 x_Triplo >= 0

_C4: - 0.25 x_Duplo - 0.25 x_Quadruplo - 0.25 x_Simples + 0.75 x_Triplo >= 0

_C5: - 0.25 x_Duplo + 0.75 x_Quadruplo - 0.25 x_Simples - 0.25 x_Triplo >= 0

_C6: 2000 n + 70000 x_Duplo + 160000 x_Quadruplo + 50000 x_Simples
 + 130000 x_Triplo <= 15000000

VARIABLES
0 <= n <= 300 Integer
0 <= x_Duplo Integer
0 <= x_Quadruplo Integer
0 <= x_Simples Integer
0 <= x_Triplo Integer

### Resolvendo o problema

In [11]:
status = modelo.solve()

### Imprimindo as soluções do problema

In [12]:
print('status: ', pulp.LpStatus[status])
print(f'funcao objetivo: ${modelo.objective.value()} impostos arrecadados')

for i in D:
  print(f'Domicílio ({i}): {x[i].value()} construídos')

status:  Optimal
funcao objetivo: $321300.0 impostos arrecadados
Domicílio (Simples): 28.0 construídos
Domicílio (Duplo): 42.0 construídos
Domicílio (Triplo): 35.0 construídos
Domicílio (Quadruplo): 35.0 construídos
